In [ ]:
!pip install pandas
!pip install numpy

In [ ]:
# Подготовка соответствующих библиотек
import pandas as pd
import numpy as np

In [ ]:
# Загрузка данных
url='https://drive.google.com/uc?id=133JIk2zLUOYdlm5wfdzXkHiUfcx8mb3A&export=download'
dataset=pd.read_csv(url)
# Для наглядности посмотрим, что за информацию получили
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
# Разделение данных, Используем train_test_split с соотношением 80/20
from sklearn.model_selection import train_test_split

train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
print(f"Train: {train_dataset.shape}, Test: {test_dataset.shape}")

Train: (712, 12), Test: (179, 12)


In [ ]:
# Фиксируем random_state для воспроизводимости
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

Train: (712, 12), Test: (179, 12)


In [ ]:
# Базовые предсказания. Наиболее частый класс
from sklearn.metrics import accuracy_score

most_frequent = train_dataset['Survived'].mode()[0]
constant_pred = np.full(len(test_dataset), most_frequent)
accuracy = accuracy_score(test_dataset['Survived'], constant_pred)
print(f"Accuracy constant: {accuracy}")

Accuracy constant: 0.5865921787709497


In [ ]:
# Случайное предсказание
random_pred = np.random.choice([0,1], size=len(test_dataset))
random_accuracy = accuracy_score(test_dataset['Survived'], random_pred)
print(f'Random Accuracy: {random_accuracy}')

Random Accuracy: 0.5083798882681564


In [ ]:
#  Интерпретация результатов:
# -> Константная модель (58.7% точности) показывает, что большинство пассажиров в тренировочных данных не выжили.
# -> Случайное предсказание близко к теоретическому значению 50%, что подтверждает сбалансированность классов.
# Эти результаты устанавливают базовый уровень (baseline) которые можно использовать при дальнейшем построении более сложных моделей.

In [ ]:
# Предобработка данных и обучение модели

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Учет особенностей датасета (названия столбцов)
numeric_features = ['Age', 'SibSp', 'Parch', 'Fare']
categoric_features = ['Pclass', 'Sex']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder())
        ]), categoric_features)
    ])

model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Age', 'SibSp', 'Parch',
                                                   'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder())]),
                                                  ['Pclass', 'Sex'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [50]:
# Оценка качества.

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.82      0.85      0.84       105
           1       0.77      0.74      0.76        74

    accuracy                           0.80       179
   macro avg       0.80      0.80      0.80       179
weighted avg       0.80      0.80      0.80       179



In [ ]:
# Точность модели (0.80) превосходит константные предсказания (0.58 и 0.50).
# Модель демонстрирует сбалансированное качество для обоих классов, что подтверждается значениями precision и recall